In [26]:
"""
AIRLINE CUSTOMER ANALYTICS - BUSINESS INTELLIGENCE FOCUS
Customer Behavior Analysis and Risk Assessment for Revenue Optimization
Author: Hansel Liebrata
Date: January 2025
Portfolio Project for Business Intelligence/Data Analyst Positions
"""

'\nAIRLINE CUSTOMER ANALYTICS - BUSINESS INTELLIGENCE FOCUS\nCustomer Behavior Analysis and Risk Assessment for Revenue Optimization\nAuthor: Hansel Liebrata\nDate: January 2025\nPortfolio Project for Business Intelligence/Data Analyst Positions\n'

# =============================================================================
# IMPORT LIBRARIES AND SETUP
# =============================================================================

In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import warnings
import os
warnings.filterwarnings('ignore')

# Configure visualization settings
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

print("AIRLINE CUSTOMER ANALYTICS - BUSINESS INTELLIGENCE PROJECT")
print("Focus: Customer Behavior Analysis & Revenue Optimization")
print("="*70)
print("Libraries imported successfully")
print("Visualization style configured")

AIRLINE CUSTOMER ANALYTICS - BUSINESS INTELLIGENCE PROJECT
Focus: Customer Behavior Analysis & Revenue Optimization
Libraries imported successfully
Visualization style configured


# =============================================================================
# LOAD AND EXAMINE DATA
# =============================================================================

In [28]:
print("\nLOADING AIRLINE BOOKING DATA...")

# Load the dataset
try:
    df = pd.read_csv('data/airline_bookings.csv', encoding='ISO-8859-1')
    print(f"Data loaded successfully: {df.shape[0]:,} bookings, {df.shape[1]} features")
except FileNotFoundError:
    print("Error: Please ensure 'airline_bookings.csv' is in the 'data/' folder")
    print("Note: Adjust the file path as needed for your setup")

# Display dataset overview
print(f"\nDATASET OVERVIEW:")
print(f"   Rows: {df.shape[0]:,}")
print(f"   Columns: {df.shape[1]}")
print(f"   Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.1f} MB")

print(f"\nCOLUMN INFORMATION:")
for i, col in enumerate(df.columns, 1):
    dtype = df[col].dtype
    unique_vals = df[col].nunique()
    print(f"   {i:2d}. {col:<25} | {str(dtype):<10} | {unique_vals:>6} unique values")


LOADING AIRLINE BOOKING DATA...
Data loaded successfully: 50,000 bookings, 14 features

DATASET OVERVIEW:
   Rows: 50,000
   Columns: 14
   Memory usage: 18.6 MB

COLUMN INFORMATION:
    1. num_passengers            | int64      |      9 unique values
    2. sales_channel             | object     |      2 unique values
    3. trip_type                 | object     |      3 unique values
    4. purchase_lead             | int64      |    470 unique values
    5. length_of_stay            | int64      |    335 unique values
    6. flight_hour               | int64      |     24 unique values
    7. flight_day                | object     |      7 unique values
    8. route                     | object     |    799 unique values
    9. booking_origin            | object     |    104 unique values
   10. wants_extra_baggage       | int64      |      2 unique values
   11. wants_preferred_seat      | int64      |      2 unique values
   12. wants_in_flight_meals     | int64      |      2 un

# =============================================================================
# DATA QUALITY ASSESSMENT
# =============================================================================

In [29]:
print("\n" + "="*70)
print("DATA QUALITY ASSESSMENT")
print("="*70)

# Check data quality
missing_data = df.isnull().sum()
total_missing = missing_data.sum()

print("DATA QUALITY REPORT:")
if total_missing > 0:
    print("   Missing values detected:")
    for col, missing in missing_data[missing_data > 0].items():
        missing_pct = (missing / len(df)) * 100
        print(f"   - {col}: {missing:,} ({missing_pct:.1f}%)")
else:
    print("   No missing values detected")

duplicates = df.duplicated().sum()
print(f"   Duplicate rows: {duplicates:,}")

# Analyze target variable
print(f"\nBUSINESS METRICS ANALYSIS:")
if 'booking_complete' in df.columns:
    total_bookings = len(df)
    
    # Handle potential label confusion (0/1 interpretation)
    completion_values = df['booking_complete'].value_counts().sort_index()
    print(f"   Booking completion distribution: {completion_values.to_dict()}")
    
    # Determine correct interpretation based on distribution
    if completion_values.get(1, 0) < completion_values.get(0, 0):
        # If 1s are fewer than 0s, likely 1=completed, 0=abandoned
        completed_bookings = df['booking_complete'].sum()
        completion_rate = df['booking_complete'].mean()
        print("   Interpretation: 1=Completed, 0=Abandoned")
    else:
        # If 0s are fewer than 1s, likely 0=completed, 1=abandoned  
        completed_bookings = (df['booking_complete'] == 0).sum()
        completion_rate = (df['booking_complete'] == 0).mean()
        print("   Interpretation: 0=Completed, 1=Abandoned")
        # Flip labels for intuitive analysis
        df['booking_complete'] = 1 - df['booking_complete']
        print("   Labels corrected: 1=Completed, 0=Abandoned")
    
    abandoned_bookings = total_bookings - completed_bookings
    
    print(f"   Total Bookings: {total_bookings:,}")
    print(f"   Completed Bookings: {completed_bookings:,} ({completion_rate:.1%})")
    print(f"   Abandoned Bookings: {abandoned_bookings:,} ({1-completion_rate:.1%})")
    
    # Business impact calculation
    avg_booking_value = 450  # Industry average
    revenue_at_risk = abandoned_bookings * avg_booking_value
    print(f"   Revenue at Risk: ${revenue_at_risk:,.0f}")



DATA QUALITY ASSESSMENT
DATA QUALITY REPORT:
   No missing values detected
   Duplicate rows: 719

BUSINESS METRICS ANALYSIS:
   Booking completion distribution: {0: 42522, 1: 7478}
   Interpretation: 1=Completed, 0=Abandoned
   Total Bookings: 50,000
   Completed Bookings: 7,478 (15.0%)
   Abandoned Bookings: 42,522 (85.0%)
   Revenue at Risk: $19,134,900


# =============================================================================
# FEATURE ENGINEERING
# =============================================================================

In [30]:
# =============================================================================
# FEATURE ENGINEERING - FIXED IMPLEMENTATION
# =============================================================================

print("\n" + "="*70)
print("FEATURE ENGINEERING")
print("="*70)

# Data cleaning
print("DATA CLEANING:")
original_shape = df.shape
df = df.dropna()
print(f"   Original records: {original_shape[0]:,}")
print(f"   After cleaning: {df.shape[0]:,} records")

print("\nCREATING BUSINESS FEATURES:")

# 1. Premium services count
df['premium_services_count'] = (df['wants_extra_baggage'] + 
                               df['wants_preferred_seat'] + 
                               df['wants_in_flight_meals'])

# FIXED: 2. Booking timing categories - handle extreme values properly
print("Creating booking timing categories...")

# First, analyze the purchase_lead distribution
print(f"Purchase lead statistics:")
print(f"   Min: {df['purchase_lead'].min()} days")
print(f"   Max: {df['purchase_lead'].max()} days") 
print(f"   Median: {df['purchase_lead'].median()} days")
print(f"   95th percentile: {df['purchase_lead'].quantile(0.95)} days")

# Identify extreme values
extreme_values = df[df['purchase_lead'] > 365]
print(f"   Records with >365 days lead: {len(extreme_values)}")

# FIXED: Create business-relevant timing categories with proper edge handling
# Cap extreme lead times at business-reasonable maximum (1 year)
df['purchase_lead_business'] = df['purchase_lead'].clip(upper=365)

# Create timing categories with clear, non-overlapping bins
df['booking_timing_category'] = pd.cut(
    df['purchase_lead_business'], 
    bins=[-0.001, 1, 7, 30, 90, 365],  # Slightly negative lower bound to include 0
    labels=['Same_Day', 'Last_Minute', 'Short_Term', 'Medium_Term', 'Long_Term'],
    include_lowest=True,
    right=True  # Right edge inclusive: (0,1], (1,7], etc.
)

# Verify no missing categories
missing_timing = df['booking_timing_category'].isnull().sum()
if missing_timing > 0:
    print(f"⚠️  WARNING: {missing_timing} records still have missing timing categories")
    # Debug the issue
    problematic = df[df['booking_timing_category'].isnull()]['purchase_lead_business'].describe()
    print("Problematic values:", problematic)
else:
    print("✅ All records have timing categories assigned")

# Show distribution
timing_dist = df['booking_timing_category'].value_counts()
print("Booking timing distribution:")
for category, count in timing_dist.items():
    pct = count / len(df) * 100
    print(f"   {category}: {count:,} ({pct:.1f}%)")

# 3. Customer value score
df['customer_value_score'] = (df['num_passengers'] * 100 + 
                             df['premium_services_count'] * 50 + 
                             np.where(df['flight_duration'] > 0, df['flight_duration']/60 * 10, 0))

# 4. Flight characteristics  
df['is_weekend_flight'] = df['flight_day'].isin(['Sat', 'Sun'])
df['is_peak_hour'] = df['flight_hour'].between(7, 9) | df['flight_hour'].between(17, 19)

print("\n✅ Business features created successfully:")
print("   - premium_services_count (0-3 services)")
print("   - booking_timing_category (5 categories, no nulls)")
print("   - customer_value_score (composite metric)")
print("   - is_weekend_flight (boolean)")
print("   - is_peak_hour (boolean)")

# Clean up temporary column (optional)
df = df.drop('purchase_lead_business', axis=1)

print(f"\n📊 FEATURE ENGINEERING SUMMARY:")
print(f"   Total features created: 5")
print(f"   Records processed: {len(df):,}")
print(f"   Data quality: ✅ Ready for analysis")


FEATURE ENGINEERING
DATA CLEANING:
   Original records: 50,000
   After cleaning: 50,000 records

CREATING BUSINESS FEATURES:
Creating booking timing categories...
Purchase lead statistics:
   Min: 0 days
   Max: 867 days
   Median: 51.0 days
   95th percentile: 287.0 days
   Records with >365 days lead: 838
✅ All records have timing categories assigned
Booking timing distribution:
   Medium_Term: 17,015 (34.0%)
   Long_Term: 15,846 (31.7%)
   Short_Term: 12,326 (24.7%)
   Last_Minute: 3,758 (7.5%)
   Same_Day: 1,055 (2.1%)

✅ Business features created successfully:
   - premium_services_count (0-3 services)
   - booking_timing_category (5 categories, no nulls)
   - customer_value_score (composite metric)
   - is_weekend_flight (boolean)
   - is_peak_hour (boolean)

📊 FEATURE ENGINEERING SUMMARY:
   Total features created: 5
   Records processed: 50,000
   Data quality: ✅ Ready for analysis


# =============================================================================
# CUSTOMER SEGMENTATION
# =============================================================================

In [31]:
# =============================================================================
# CUSTOMER SEGMENTATION ANALYSIS - FIXED IMPLEMENTATION
# =============================================================================

print("\n" + "="*70)
print("CUSTOMER SEGMENTATION ANALYSIS")
print("="*70)

print("IDENTIFYING CUSTOMER SEGMENTS...")

# Prepare features for clustering
clustering_features = ['num_passengers', 'purchase_lead', 'length_of_stay', 
                      'premium_services_count', 'customer_value_score', 'flight_duration']

# Create clean clustering dataset
df_clustering = df[clustering_features].copy()
df_clustering = df_clustering.fillna(df_clustering.median())

# FIXED: Handle outliers properly - cap at 95th percentile for business relevance
print("Handling outliers...")
for col in clustering_features:
    q95 = df_clustering[col].quantile(0.95)
    q05 = df_clustering[col].quantile(0.05)
    print(f"  {col}: capping at {q05:.1f} - {q95:.1f}")
    df_clustering[col] = df_clustering[col].clip(lower=q05, upper=q95)

# Standardize features
scaler = StandardScaler()
features_scaled = scaler.fit_transform(df_clustering)

# Perform clustering
n_clusters = 4
kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
df['customer_segment'] = kmeans.fit_predict(features_scaled)

# Analyze segment characteristics
print("\nSEGMENT CHARACTERISTICS:")
segment_analysis = df.groupby('customer_segment').agg({
    'booking_complete': ['count', 'mean'],
    'num_passengers': 'mean',
    'purchase_lead': 'mean',
    'premium_services_count': 'mean',
    'customer_value_score': 'mean'
}).round(2)

segment_analysis.columns = ['Bookings', 'Completion_Rate', 'Avg_Passengers', 
                           'Avg_Lead_Days', 'Avg_Premium_Services', 'Avg_Value_Score']

print(segment_analysis.to_string())

# FIXED: More robust segment naming logic with hierarchical rules and fallbacks
print("\nApplying intelligent segment naming...")

segment_names = {}
used_names = set()

# Define segment naming rules in priority order
def assign_segment_name(segment_data):
    """Assign segment name based on characteristics with priority rules"""
    
    # Rule 1: Premium Business Travelers (highest priority - revenue)
    if (segment_data['Avg_Premium_Services'] >= 1.2 and 
        segment_data['Avg_Value_Score'] > segment_analysis['Avg_Value_Score'].median()):
        return "Premium_Business_Travelers"
    
    # Rule 2: Family Groups (large groups regardless of other factors)
    elif segment_data['Avg_Passengers'] >= 2.5:
        return "Family_Groups"
    
    # Rule 3: Last Minute Bookers (short lead time)
    elif segment_data['Avg_Lead_Days'] <= 14:
        return "Last_Minute_Bookers"
    
    # Rule 4: Budget Conscious (default for remaining)
    else:
        return "Budget_Conscious_Travelers"

# Apply naming with conflict resolution
for i in range(n_clusters):
    segment_data = segment_analysis.iloc[i]
    proposed_name = assign_segment_name(segment_data)
    
    # Handle naming conflicts by adding distinguishing characteristics
    if proposed_name in used_names:
        if proposed_name == "Premium_Business_Travelers":
            if segment_data['Avg_Passengers'] >= 2.5:
                proposed_name = "Premium_Family_Travelers"
            else:
                proposed_name = "Premium_Solo_Travelers"
        elif proposed_name == "Budget_Conscious_Travelers":
            if segment_data['Avg_Lead_Days'] <= 30:
                proposed_name = "Budget_Short_Term_Bookers"
            else:
                proposed_name = "Budget_Long_Term_Planners"
        elif proposed_name == "Last_Minute_Bookers":
            if segment_data['Avg_Premium_Services'] >= 0.5:
                proposed_name = "Last_Minute_Premium_Bookers"
            else:
                proposed_name = "Last_Minute_Budget_Bookers"
        elif proposed_name == "Family_Groups":
            if segment_data['Avg_Premium_Services'] >= 1.0:
                proposed_name = "Premium_Family_Groups"
            else:
                proposed_name = "Budget_Family_Groups"
    
    segment_names[i] = proposed_name
    used_names.add(proposed_name)
    
    print(f"  Segment {i}: {proposed_name}")
    print(f"    Passengers: {segment_data['Avg_Passengers']:.1f}, "
          f"Lead Days: {segment_data['Avg_Lead_Days']:.0f}, "
          f"Premium: {segment_data['Avg_Premium_Services']:.1f}, "
          f"Value: {segment_data['Avg_Value_Score']:.0f}")

# Apply segment names to dataframe
df['segment_name'] = df['customer_segment'].map(segment_names)

# VERIFICATION: Ensure all segments have names
missing_names = df['segment_name'].isnull().sum()
if missing_names > 0:
    print(f"WARNING: {missing_names} records without segment names!")
    # Emergency fallback
    df['segment_name'] = df['segment_name'].fillna('Unclassified_Segment')
else:
    print("✅ All segments successfully named")

# FIXED: Comprehensive segment profiling
print(f"\nBUSINESS SEGMENT PROFILES:")
print("="*50)

for segment in range(n_clusters):
    if segment in segment_names:
        name = segment_names[segment]
        segment_data = segment_analysis.loc[segment]
        segment_size = len(df[df['customer_segment'] == segment])
        segment_pct = segment_size / len(df) * 100
        completion_rate = segment_data['Completion_Rate']
        value_score = segment_data['Avg_Value_Score']
        
        print(f"\n📊 SEGMENT {segment}: {name}")
        print(f"   Size: {segment_size:,} customers ({segment_pct:.1f}%)")
        print(f"   Completion Rate: {completion_rate:.1%}")
        print(f"   Avg Group Size: {segment_data['Avg_Passengers']:.1f} passengers")
        print(f"   Avg Lead Time: {segment_data['Avg_Lead_Days']:.0f} days")
        print(f"   Premium Services: {segment_data['Avg_Premium_Services']:.1f}")
        print(f"   Value Score: {value_score:.0f}")
        
        # Enhanced business recommendations
        if completion_rate < 0.65:
            recommendation = "🚨 URGENT: High intervention priority"
            priority = "HIGH"
        elif completion_rate < 0.75:
            recommendation = "⚠️  MONITOR: Targeted retention needed"
            priority = "MEDIUM"
        elif completion_rate > 0.85:
            recommendation = "✅ MAINTAIN: Excellent performance"
            priority = "LOW"
        else:
            recommendation = "📈 OPTIMIZE: Good but improvable"
            priority = "MEDIUM"
            
        print(f"   Strategy: {recommendation}")
        print(f"   Priority: {priority}")

# Final verification
print(f"\n" + "="*70)
print("SEGMENTATION SUMMARY:")
print(f"✅ Total Segments Created: {len(segment_names)}")
print(f"✅ All Segments Named: {missing_names == 0}")
print(f"✅ Unique Names: {len(set(segment_names.values())) == len(segment_names)}")
segment_distribution = df['segment_name'].value_counts()
print(f"✅ Segment Distribution:")
for name, count in segment_distribution.items():
    pct = count / len(df) * 100
    print(f"   {name}: {count:,} ({pct:.1f}%)")
print("="*70)


CUSTOMER SEGMENTATION ANALYSIS
IDENTIFYING CUSTOMER SEGMENTS...
Handling outliers...
  num_passengers: capping at 1.0 - 4.0
  purchase_lead: capping at 4.0 - 287.0
  length_of_stay: capping at 3.0 - 84.0
  premium_services_count: capping at 0.0 - 3.0
  customer_value_score: capping at 101.1 - 451.4
  flight_duration: capping at 4.7 - 8.8

SEGMENT CHARACTERISTICS:
                  Bookings  Completion_Rate  Avg_Passengers  Avg_Lead_Days  Avg_Premium_Services  Avg_Value_Score
customer_segment                                                                                                 
0                    15835             0.16            1.44          91.79                  2.30           260.04
1                     5709             0.14            1.20          57.55                  1.68           205.46
2                    22658             0.14            1.21          76.31                  0.64           154.29
3                     5798             0.17            3.87    

# =============================================================================
# BUSINESS RISK SCORING
# =============================================================================

In [32]:
print("\n" + "="*70)
print("BUSINESS RISK SCORING SYSTEM")
print("="*70)

print("DEVELOPING PRACTICAL RISK ASSESSMENT...")

def calculate_business_risk_score(row):
    """
    Calculate risk score using transparent business logic
    Scale: 0-100 (higher = more likely to abandon)
    """
    risk_score = 0
    
    # Booking timing risk (40 points max)
    if row['purchase_lead'] <= 1:
        risk_score += 40  # Same day = highest risk
    elif row['purchase_lead'] <= 7:
        risk_score += 25  # Last minute = high risk
    elif row['purchase_lead'] <= 30:
        risk_score += 10  # Short term = medium risk
    else:
        risk_score += 5   # Advance = lowest risk
        
    # Premium services risk (30 points max)
    if row['premium_services_count'] == 0:
        risk_score += 30  # No services = high risk
    elif row['premium_services_count'] == 1:
        risk_score += 15  # Some services = medium risk
    else:
        risk_score += 5   # Multiple services = low risk
        
    # Sales channel risk (20 points max)
    channel_risk = {
        'Online': 20,      # Highest abandonment
        'Mobile': 15,      # Medium risk
        'Travel Agent': 5, # Lowest risk
        'Phone': 8         # Low-medium risk
    }
    risk_score += channel_risk.get(row['sales_channel'], 10)
        
    # Group dynamics risk (10 points max)
    if row['num_passengers'] == 1:
        risk_score += 10  # Solo = higher risk
    elif row['num_passengers'] >= 4:
        risk_score += 3   # Large groups = low risk
    else:
        risk_score += 6   # Small groups = medium risk
        
    return min(risk_score, 100)

# Apply risk scoring
print("Applying business risk scoring...")
df['risk_score'] = df.apply(calculate_business_risk_score, axis=1)

# Create risk categories
df['risk_category'] = pd.cut(df['risk_score'], 
                            bins=[0, 30, 60, 100], 
                            labels=['Low_Risk', 'Medium_Risk', 'High_Risk'],
                            include_lowest=True)

print("RISK SCORING COMPLETED")

# Validate risk scoring effectiveness
print("\nRISK SCORE VALIDATION:")
risk_validation = df.groupby('risk_category')['booking_complete'].agg(['count', 'mean'])
print("Risk Category | Bookings | Actual Completion Rate | Expected Result")
print("--------------|----------|----------------------|----------------")
for category, data in risk_validation.iterrows():
    if str(category) == 'High_Risk':
        expected = "Low completion rate"
    elif str(category) == 'Low_Risk':
        expected = "High completion rate"
    else:
        expected = "Medium completion rate"
    print(f"{str(category):<13} | {data['count']:>8,} | {data['mean']:>19.1%} | {expected}")

# Risk score distribution
print(f"\nRISK SCORE DISTRIBUTION:")
print(f"  Average risk score: {df['risk_score'].mean():.1f}")
print(f"  Risk score range: {df['risk_score'].min():.0f} - {df['risk_score'].max():.0f}")

# Business logic explanation
print(f"\nRISK SCORING METHODOLOGY:")
print("  Booking Timing (40%): Same-day bookings = highest risk")
print("  Premium Services (30%): No services = higher abandonment")
print("  Sales Channel (20%): Online bookings = higher risk")
print("  Group Size (10%): Solo travelers = higher risk")
print("  Total possible score: 0-100 (higher = more likely to abandon)")


BUSINESS RISK SCORING SYSTEM
DEVELOPING PRACTICAL RISK ASSESSMENT...
Applying business risk scoring...
RISK SCORING COMPLETED

RISK SCORE VALIDATION:
Risk Category | Bookings | Actual Completion Rate | Expected Result
--------------|----------|----------------------|----------------
Low_Risk      | 13,280.0 |               16.6% | High completion rate
Medium_Risk   | 34,359.0 |               14.5% | Medium completion rate
High_Risk     |  2,361.0 |               12.1% | Low completion rate

RISK SCORE DISTRIBUTION:
  Average risk score: 41.2
  Risk score range: 23 - 95

RISK SCORING METHODOLOGY:
  Booking Timing (40%): Same-day bookings = highest risk
  Premium Services (30%): No services = higher abandonment
  Sales Channel (20%): Online bookings = higher risk
  Group Size (10%): Solo travelers = higher risk
  Total possible score: 0-100 (higher = more likely to abandon)


# =============================================================================
# BUSINESS IMPACT ANALYSIS
# =============================================================================

In [33]:
print("\n" + "="*70)
print("BUSINESS IMPACT ANALYSIS")
print("="*70)

# Current business metrics
total_bookings = len(df)
completed_bookings = df['booking_complete'].sum()
abandoned_bookings = total_bookings - completed_bookings
completion_rate = df['booking_complete'].mean()
avg_booking_value = 450

current_revenue = completed_bookings * avg_booking_value
revenue_at_risk = abandoned_bookings * avg_booking_value

print(f"CURRENT BUSINESS PERFORMANCE:")
print(f"  Total Bookings: {total_bookings:,}")
print(f"  Completed Bookings: {completed_bookings:,} ({completion_rate:.1%})")
print(f"  Abandoned Bookings: {abandoned_bookings:,} ({1-completion_rate:.1%})")
print(f"  Current Revenue: ${current_revenue:,.0f}")
print(f"  Revenue at Risk: ${revenue_at_risk:,.0f}")

# Risk-based intervention analysis
high_risk_bookings = df[df['risk_category'] == 'High_Risk']
medium_risk_bookings = df[df['risk_category'] == 'Medium_Risk']
low_risk_bookings = df[df['risk_category'] == 'Low_Risk']

print(f"\nRISK-BASED INTERVENTION OPPORTUNITIES:")
print(f"  High Risk: {len(high_risk_bookings):,} bookings ({len(high_risk_bookings)/len(df):.1%})")
print(f"  Medium Risk: {len(medium_risk_bookings):,} bookings ({len(medium_risk_bookings)/len(df):.1%})")
print(f"  Low Risk: {len(low_risk_bookings):,} bookings ({len(low_risk_bookings)/len(df):.1%})")

# Intervention strategy economics
intervention_cost_per_booking = 15  # Email/SMS/call cost
recovery_success_rate = 0.25  # Conservative success rate

# Focus on high-risk bookings
target_bookings = len(high_risk_bookings)
total_intervention_cost = target_bookings * intervention_cost_per_booking
expected_recoveries = target_bookings * recovery_success_rate
recovery_revenue = expected_recoveries * avg_booking_value
net_benefit = recovery_revenue - total_intervention_cost
roi_ratio = recovery_revenue / total_intervention_cost if total_intervention_cost > 0 else 0

print(f"\nINTERVENTION STRATEGY (HIGH-RISK FOCUS):")
print(f"  Target Population: {target_bookings:,} bookings")
print(f"  Intervention Cost: ${total_intervention_cost:,.0f}")
print(f"  Expected Recoveries: {expected_recoveries:.0f} bookings")
print(f"  Recovery Revenue: ${recovery_revenue:,.0f}")
print(f"  Net Benefit: ${net_benefit:,.0f}")
print(f"  ROI: {roi_ratio:.1f}x return on investment")

# Annual projection
quarters_per_year = 4
annual_net_benefit = net_benefit * quarters_per_year

print(f"\nANNUAL BUSINESS IMPACT:")
print(f"  Projected Annual Benefit: ${annual_net_benefit:,.0f}")
print(f"  Annual Investment Required: ${total_intervention_cost * quarters_per_year:,.0f}")

# Segment-specific opportunities
print(f"\nSEGMENT-SPECIFIC OPPORTUNITIES:")
segment_opportunities = df.groupby('segment_name').agg({
    'booking_complete': ['count', 'mean'],
    'risk_score': 'mean'
}).round(2)

segment_opportunities.columns = ['Bookings', 'Completion_Rate', 'Avg_Risk_Score']

for segment, data in segment_opportunities.iterrows():
    opportunity_size = data['Bookings'] * (1 - data['Completion_Rate']) * avg_booking_value
    print(f"  {segment}:")
    print(f"    Revenue at risk: ${opportunity_size:,.0f}")
    print(f"    Average risk score: {data['Avg_Risk_Score']:.0f}")



BUSINESS IMPACT ANALYSIS
CURRENT BUSINESS PERFORMANCE:
  Total Bookings: 50,000
  Completed Bookings: 7,478 (15.0%)
  Abandoned Bookings: 42,522 (85.0%)
  Current Revenue: $3,365,100
  Revenue at Risk: $19,134,900

RISK-BASED INTERVENTION OPPORTUNITIES:
  High Risk: 2,361 bookings (4.7%)
  Medium Risk: 34,359 bookings (68.7%)
  Low Risk: 13,280 bookings (26.6%)

INTERVENTION STRATEGY (HIGH-RISK FOCUS):
  Target Population: 2,361 bookings
  Intervention Cost: $35,415
  Expected Recoveries: 590 bookings
  Recovery Revenue: $265,612
  Net Benefit: $230,198
  ROI: 7.5x return on investment

ANNUAL BUSINESS IMPACT:
  Projected Annual Benefit: $920,790
  Annual Investment Required: $141,660

SEGMENT-SPECIFIC OPPORTUNITIES:
  Budget_Conscious_Travelers:
    Revenue at risk: $2,209,383
    Average risk score: 40
  Budget_Long_Term_Planners:
    Revenue at risk: $8,768,646
    Average risk score: 50
  Premium_Business_Travelers:
    Revenue at risk: $5,985,630
    Average risk score: 33
  Prem

# =============================================================================
# KEY INSIGHTS AND RECOMMENDATIONS
# =============================================================================

In [34]:
print("\n" + "="*70)
print("EXECUTIVE SUMMARY & RECOMMENDATIONS")
print("="*70)

print("KEY BUSINESS INSIGHTS:")

# 1. Premium services impact
premium_impact = df.groupby('premium_services_count')['booking_complete'].mean()
premium_uplift = premium_impact.max() - premium_impact.min()
print(f"\n1. PREMIUM SERVICES OPPORTUNITY:")
print(f"   • Completion rate ranges from {premium_impact.min():.1%} to {premium_impact.max():.1%}")
print(f"   • Potential uplift: {premium_uplift:.1%} through premium service promotion")
print(f"   • Revenue impact: ${len(df) * premium_uplift * avg_booking_value:,.0f} if applied to all bookings")

# 2. Timing optimization
timing_performance = df.groupby('booking_timing_category')['booking_complete'].mean()
best_timing = timing_performance.idxmax()
worst_timing = timing_performance.idxmin()
print(f"\n2. BOOKING TIMING OPTIMIZATION:")
print(f"   • Best window: {best_timing} ({timing_performance[best_timing]:.1%} completion)")
print(f"   • Worst window: {worst_timing} ({timing_performance[worst_timing]:.1%} completion)")
print(f"   • Strategy: Target marketing during optimal booking windows")

# 3. Channel performance
channel_performance = df.groupby('sales_channel')['booking_complete'].mean()
best_channel = channel_performance.idxmax()
worst_channel = channel_performance.idxmin()
channel_gap = channel_performance.max() - channel_performance.min()
print(f"\n3. SALES CHANNEL OPTIMIZATION:")
print(f"   • Best channel: {best_channel} ({channel_performance[best_channel]:.1%})")
print(f"   • Worst channel: {worst_channel} ({channel_performance[worst_channel]:.1%})")
print(f"   • Improvement opportunity: {channel_gap:.1%} through channel optimization")

# 4. Customer segments
best_segment = segment_opportunities['Completion_Rate'].idxmax()
worst_segment = segment_opportunities['Completion_Rate'].idxmin()
print(f"\n4. CUSTOMER SEGMENT INSIGHTS:")
print(f"   • Best performing: {best_segment} ({segment_opportunities.loc[best_segment, 'Completion_Rate']:.1%})")
print(f"   • Needs attention: {worst_segment} ({segment_opportunities.loc[worst_segment, 'Completion_Rate']:.1%})")

# 5. Risk scoring effectiveness
risk_effectiveness = df.groupby('risk_category')['booking_complete'].mean()
print(f"\n5. RISK SCORING VALIDATION:")
print(f"   • High-risk group completion: {risk_effectiveness['High_Risk']:.1%}")
print(f"   • Low-risk group completion: {risk_effectiveness['Low_Risk']:.1%}")
print(f"   • Risk differentiation: {(risk_effectiveness['Low_Risk'] - risk_effectiveness['High_Risk']):.1%}")

print(f"\nSTRATEGIC RECOMMENDATIONS:")
print("IMMEDIATE ACTIONS (0-30 days):")
print(f"  1. Implement risk scoring system for real-time booking assessment")
print(f"  2. Deploy intervention campaigns for {target_bookings:,} high-risk bookings")
print(f"  3. Promote premium services to medium-risk customers")
print(f"  4. Optimize {worst_channel} channel performance")

print("\nMEDIUM-TERM INITIATIVES (1-6 months):")
print("  1. Develop segment-specific retention strategies")
print("  2. Adjust marketing spend toward optimal booking windows")
print("  3. Create automated alert system for high-risk bookings")
print("  4. A/B test intervention strategies for different risk levels")

print("\nEXPECTED BUSINESS IMPACT:")
print(f"  • Immediate revenue recovery: ${net_benefit:,.0f} per quarter")
print(f"  • Annual revenue opportunity: ${annual_net_benefit:,.0f}")
print(f"  • ROI on intervention investment: {roi_ratio:.1f}x")
print(f"  • Implementation complexity: LOW (business rules-based)")



EXECUTIVE SUMMARY & RECOMMENDATIONS
KEY BUSINESS INSIGHTS:

1. PREMIUM SERVICES OPPORTUNITY:
   • Completion rate ranges from 10.7% to 18.6%
   • Potential uplift: 7.9% through premium service promotion
   • Revenue impact: $1,782,714 if applied to all bookings

2. BOOKING TIMING OPTIMIZATION:
   • Best window: Last_Minute (18.4% completion)
   • Worst window: Same_Day (13.6% completion)
   • Strategy: Target marketing during optimal booking windows

3. SALES CHANNEL OPTIMIZATION:
   • Best channel: Internet (15.5%)
   • Worst channel: Mobile (10.8%)
   • Improvement opportunity: 4.6% through channel optimization

4. CUSTOMER SEGMENT INSIGHTS:
   • Best performing: Premium_Family_Travelers (17.0%)
   • Needs attention: Budget_Conscious_Travelers (14.0%)

5. RISK SCORING VALIDATION:
   • High-risk group completion: 12.1%
   • Low-risk group completion: 16.6%
   • Risk differentiation: 4.5%

STRATEGIC RECOMMENDATIONS:
IMMEDIATE ACTIONS (0-30 days):
  1. Implement risk scoring system for

# =============================================================================
# DATA EXPORT FOR POWER BI
# =============================================================================

In [35]:
# =============================================================================
# FIXED DATA EXPORT FOR POWER BI - ENCODING ISSUE RESOLVED
# =============================================================================

import pandas as pd
import os

# Create export directory
os.makedirs('exports', exist_ok=True)

print("EXPORTING BUSINESS INTELLIGENCE DATASETS...")

# 1. Main dataset with all analysis
df_export = df.copy()
df_export['analysis_date'] = pd.Timestamp.now().strftime('%Y-%m-%d')

# 2. Customer segment summary
segment_summary = df.groupby(['customer_segment', 'segment_name']).agg({
    'booking_complete': ['count', 'mean'],
    'risk_score': 'mean',
    'premium_services_count': 'mean',
    'customer_value_score': 'mean'
}).round(3)

segment_summary.columns = ['Total_Bookings', 'Completion_Rate', 'Avg_Risk_Score', 
                          'Avg_Premium_Services', 'Avg_Value_Score']
segment_summary = segment_summary.reset_index()

# 3. Sales channel performance
channel_summary = df.groupby('sales_channel').agg({
    'booking_complete': ['count', 'mean'],
    'risk_score': 'mean',
    'premium_services_count': 'mean'
}).round(3)

channel_summary.columns = ['Total_Bookings', 'Completion_Rate', 'Avg_Risk_Score', 'Avg_Premium_Services']
channel_summary = channel_summary.reset_index()

# 4. Business metrics summary
business_metrics = {
    'total_bookings': len(df),
    'completion_rate': df['booking_complete'].mean(),
    'revenue_at_risk': abandoned_bookings * avg_booking_value,
    'high_risk_bookings': len(high_risk_bookings),
    'intervention_roi': roi_ratio,
    'annual_opportunity': annual_net_benefit,
    'avg_risk_score': df['risk_score'].mean(),
    'best_segment': best_segment,
    'best_channel': best_channel,
    'optimal_timing': best_timing,
    'analysis_date': pd.Timestamp.now().strftime('%Y-%m-%d')
}

business_metrics_df = pd.DataFrame([business_metrics])

# Export all datasets
print("SAVING POWER BI DATASETS:")

# Main dataset
df_export.to_csv('exports/airline_bookings_complete_analysis.csv', index=False)
print(f"  ✓ airline_bookings_complete_analysis.csv ({len(df_export):,} rows)")

# Summary tables
segment_summary.to_csv('exports/customer_segment_summary.csv', index=False)
print(f"  ✓ customer_segment_summary.csv ({len(segment_summary)} segments)")

channel_summary.to_csv('exports/sales_channel_summary.csv', index=False)
print(f"  ✓ sales_channel_summary.csv ({len(channel_summary)} channels)")

business_metrics_df.to_csv('exports/business_metrics_summary.csv', index=False)
print(f"  ✓ business_metrics_summary.csv (executive KPIs)")

# Create data dictionary
data_dictionary = pd.DataFrame({
    'Column_Name': [
        'num_passengers', 'sales_channel', 'trip_type', 'purchase_lead', 'length_of_stay',
        'flight_hour', 'flight_day', 'route', 'booking_origin', 'wants_extra_baggage',
        'wants_preferred_seat', 'wants_in_flight_meals', 'flight_duration', 'booking_complete',
        'premium_services_count', 'booking_timing_category', 'customer_value_score',
        'is_weekend_flight', 'is_peak_hour', 'customer_segment', 'segment_name',
        'risk_score', 'risk_category', 'analysis_date'
    ],
    'Description': [
        'Number of passengers in booking (1-8)',
        'Channel used for booking (Online, Mobile, Agent, Phone)',
        'Trip type (Round Trip, One Way)',
        'Days between booking and departure (0-365)',
        'Length of stay at destination in days',
        'Flight departure hour (0-23)',
        'Day of week for flight departure',
        'Flight route code (origin-destination)',
        'Country where booking was made',
        'Customer selected extra baggage (0/1)',
        'Customer selected preferred seat (0/1)',
        'Customer selected in-flight meals (0/1)',
        'Flight duration in minutes',
        'Booking completion status (1=completed, 0=abandoned)',
        'Total premium services selected (0-3)',
        'Booking timing category (Same_Day to Long_Term)',
        'Composite customer value score (0-500+)',
        'Flight scheduled for weekend (True/False)',
        'Flight during peak hours 7-9am or 5-7pm (True/False)',
        'Customer segment number (0-3)',
        'Business name for customer segment',
        'Risk score for booking abandonment (0-100)',
        'Risk category (Low/Medium/High Risk)',
        'Date when analysis was performed'
    ],
    'Business_Use': [
        'Group size analysis, family vs business travel',
        'Channel performance optimization',
        'Trip complexity impact on completion',
        'Booking window optimization',
        'Trip duration impact analysis',
        'Peak hour performance analysis',
        'Seasonal demand patterns',
        'Route performance analysis',
        'Geographic market analysis',
        'Premium service adoption analysis',
        'Seat preference impact',
        'Meal service impact',
        'Flight length impact on completion',
        'PRIMARY TARGET METRIC - Revenue optimization',
        'Premium service bundle analysis',
        'Marketing campaign timing',
        'Customer lifetime value estimation',
        'Operational planning',
        'Resource allocation optimization',
        'Segmentation strategy',
        'Targeted marketing campaigns',
        'INTERVENTION PRIORITIZATION - Risk management',
        'Intervention strategy deployment',
        'Data freshness tracking'
    ]
})

data_dictionary.to_csv('exports/data_dictionary.csv', index=False)
print(f"  ✓ data_dictionary.csv (column definitions)")

print(f"\nEXPORT SUMMARY:")
print(f"  📊 Total files created: 5")
print(f"  📈 Main dataset: {len(df_export):,} rows × {len(df_export.columns)} columns")
print(f"  🎯 Ready for Power BI dashboard development")
print(f"  💼 Business value: ${annual_net_benefit:,.0f} annual opportunity")

print("\n" + "="*70)
print("DATA EXPORT COMPLETED SUCCESSFULLY")
print("="*70)
print("All files exported without encoding errors!")
print("Ready for Power BI import and dashboard creation.")

EXPORTING BUSINESS INTELLIGENCE DATASETS...
SAVING POWER BI DATASETS:
  ✓ airline_bookings_complete_analysis.csv (50,000 rows)
  ✓ customer_segment_summary.csv (4 segments)
  ✓ sales_channel_summary.csv (2 channels)
  ✓ business_metrics_summary.csv (executive KPIs)
  ✓ data_dictionary.csv (column definitions)

EXPORT SUMMARY:
  📊 Total files created: 5
  📈 Main dataset: 50,000 rows × 24 columns
  🎯 Ready for Power BI dashboard development
  💼 Business value: $920,790 annual opportunity

DATA EXPORT COMPLETED SUCCESSFULLY
All files exported without encoding errors!
Ready for Power BI import and dashboard creation.
